# Task 3 (in continuation to task 2)
- Download the CSV file from the link- https://drive.google.com/file/d/1PLYwrGn5YApyWU2QpjbdhM6tea0HuGq7/view?usp=sharing
- Scrape the Twitter profile with Python Selenium or Beautiful Soup
- Details needed are - `Bio, Following Count, Followers Count, Location, Website(If available)`
- The program should insert data in MySQL database (instead of a CSV file.)

The code should be well commented and optimized, there will be extra marks for that
Make a short video explaining and running the task in <10mins. 

Don't need to show long-running codes.

Upload your code to a GitHub public repository
Submit the video and the GitHub link for the same in the submission form


Scrape the Twitter profile with Python Selenium from the links stored in csv file. Obtain `Bio, Following Count, Followers Count, Location, Website` from the profile if available.

In [5]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys


In [6]:
def launch_chrome(link, headless=False):
    ops = Options()  # create object of Options

    if headless:
        ops.add_argument('--headless')  # headless browser testing

    # prevents browser from closing when function is returned
    ops.add_experimental_option("detach", True)

    driver = webdriver.Chrome(options=ops,  service=Service(
        ChromeDriverManager().install()))

    driver.get(link)
    return driver


In [7]:
def get_element(driver, path, timeout):
    element_present = None

    try:
        element_present = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.XPATH, path)))

    except TimeoutException:
        print("Timed out waiting for page to load and finding the required web element.")
        print(path)
    finally:
        return element_present


## Get links for csv file

In [8]:
links = []

with open('twitter_links.csv', 'r') as links_csv:
    rows = csv.reader(links_csv)
    # displaying the contents of the CSV file

    for row in rows:
        links.append(row[0])

print(links)

['https://twitter.com/GTNUK1', 'https://twitter.com/whatsapp', 'https://twitter.com/aacb_CBPTrade', 'https://twitter.com/aacbdotcom', 'https://twitter.com/@AAWindowPRODUCT', 'https://www.twitter.com/aandb_kia', 'https://twitter.com/ABHomeInc', 'https://twitter.com/Abrepro', 'http://www.twitter.com', 'https://twitter.com/ACChristofiLtd', 'https://twitter.com/aeclothing1', 'http://www.twitter.com/', 'https://twitter.com/AETechnologies1', 'http://www.twitter.com/wix', 'https://twitter.com/AGInsuranceLLC']


In [9]:
xpaths = {
    'BIO':    
    '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div/div/div/div[3]',

    'LOCATION':
    '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div/div/div/div[4]/div/span[1]/span',

    'FOLLOWING':
    '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div/div/div/div[5]/div[1]/a/span[1]/span',

    'FOLLOWERS':
    '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div/div/div/div[5]/div[2]/a/span[1]/span',

    'WEBSITE':   
    '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div/div/div/div[4]/div/a/span'
    
    }

In [54]:
def get_details(drive):
    
    bio = get_element(drive, xpaths['BIO'], timeout=10)
    location = get_element(drive, xpaths['LOCATION'], timeout=10)
    following = get_element(drive, xpaths['FOLLOWING'], timeout=10)
    followers = get_element(drive, xpaths['FOLLOWERS'], timeout=10)
    website = get_element(drive, xpaths['WEBSITE'], timeout=10)
    elements = [bio, location, following, followers, website]

    details = []
    for element in elements:
        if element:
            details.append(element.text.replace('\n', '\\n').replace('\'', ""))
            # replace is used to remove the problem occuring 
            # due to new line present (especially in bio)
            
        else:
            details.append(None)

    print(details)
    return details

## Connect to MySQL server and create database `twitter` and table `profiles`

In [11]:
from mysql import connector

In [12]:
db_creation = connector.connect(host='localhost', user='root', password='root')

In [13]:
cursor1 = db_creation.cursor()

In [14]:
cursor1.execute("CREATE DATABASE twitter")

In [15]:
cursor1.close()
db_creation.close()


In [45]:
# connecting to twitter database created above
db = connector.connect(
    host='localhost', user='root', password='root', database='twitter')
cursor = db.cursor()


**Create table**

In [46]:
cursor.execute(
    '''CREATE TABLE profiles(
            Id INT NOT NULL,
            Link VARCHAR(150) NOT NULL, 
            Bio VARCHAR(250),
            Location VARCHAR(50),
            Following VARCHAR(15),
            Followers VARCHAR(15),
            Website VARCHAR(150),
            PRIMARY KEY (Id));'''
)

In [47]:
def insert_row_in_db(details):
    Id, Link, Bio, Location, Following, Followers, Website = details

    cursor.execute(
        f'''INSERT INTO profiles (Id, Link, Bio, Location, Following, Followers, Website)
        VALUES ({Id}, '{Link}', '{Bio}', '{Location}', '{Following}', '{Followers}', '{Website}');
        '''
    )

    db.commit()

**Scrape the data and insert each row in table**

In [61]:
for i, link in enumerate(links):
    tdriver = launch_chrome(link)
    details = get_details(tdriver)
    insert_row_in_db([i+1, link]+details)
    tdriver.close()
    tdriver.quit()


['Providing Entertainment & Travel to Commercial Radio. Reaching 28.9M weekly listeners. Winners of The Arqiva National Sales Team of the Year 2010, 2011 & 2016', 'London, England', '463', '126', 'gtn.uk.com/index.php']
['Happy #PrideMonth \\nWith WhatsApp Channels stay in the loop on the topics you care about. Out now in Colombia  and Singapore  Coming soon ', 'California', '2', '4.9M', 'bit.ly/3WWWWXn']
Timed out waiting for page to load and finding the required web element.
//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div/div/div/div[4]/div/a/span
['Customs Broker', 'Florida, USA', '125', '31', None]
['A & A Freight | Warehousing | Customs Brokerage | Helping people ship across borders.', 'Worldwide', '4,072', '665', 'aacb.com']
['A commercial glass and glazing company serving the window industry in New England since 1954. #SafetyQualityService', 'Malden, MA', '90', '76', 'aawindowproducts.com']
['A&B Kia is a Kia dealer in Benwood, WV. Stay connected 

**Close the connection**

In [ ]:
cursor.close()
db.close()